# Evaulate uncertainty measure
- To evaluate different uncertainty measure, use AUROC. 
- Uncertainty measure expected to be higher for incorrect answers and lower for correct answers. As we want to show the relationship incorrect answer -> High uncertainty, incorrect answers are encoded with label 1, correct answers with label 0 
- Compute AUROC score for each uncertainty measure with sklearn.metrics.roc_auc_score([0, 0, 0, 1, 1, 0, ...], [semantic entropy scores of the answers])


Page 17: They compare Rouge-L > 0.3, Rouge-L > 0.5, exact matching, semantic matching 


## Predictive Entropy

The predictive entropy is defined as:
$$PE(x) = H[p(y \mid x)] = - \sum_{y} p(y \mid x) \ln p(y \mid x)$$

Since this is the expected value of $\ln p(y \mid x)$, Monte Carlo sampling approximates it by averaging log-probabilities from sampled outcomes. Thus, the Monte Carlo estimator is:
$$PE(x) \approx - \frac{1}{n}  \sum_{i = 1}^{n} \ln p(y_i \mid x)$$ where $n$ is the number of sampled answers for the question $x$.

## Number of semantically distinct answers

## Semantic Entropy

## Plots
- Bar plot: x-axis Semantic entropy, normalised entropy, entropy; y-axis: AUROC (page 2)
- 